In [5]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import ipynb.fs.full.features as features
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import make_classification
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features_distancias as f_distancias


In [6]:
df = pd.read_csv("./data/train.csv")

df = features.llenar_nulls(df)

column_names = []
for column in df.columns:
    name = column.replace(" ", "_")
    column_names.append(name)
df.columns = column_names


In [7]:
df_train, df_test = utils.dividir_df_testeo(df, test_size=0.15)

In [8]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train_f = pd.merge(df_train_f, df_train_idf, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, df_test_idf, on= 'id', how= 'left')

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

In [5]:
categoricas = ['tipodepropiedad', 'provincia', 'zona', 'intervalo_metros_cubiertos', 'intervalo_metros_totales', 'escualas_centros_cercanos']
categoricas_ohe = []

for i in categoricas:
    df_test_f, nuevas_cols = features.columna_a_ohe(df_test_f, i, df_aux=df_train_f, devolver_cols=True)
    df_train_f = features.columna_a_ohe(df_train_f, i, df_aux=df_test_f)
    categoricas_ohe += nuevas_cols

In [10]:
features = list(df_train_f.columns)

features_drop = ['id', 'titulo', 'descripcion', 'direccion', 'intervalo_metros_cubiertos', 
                 'intervalo_metros_totales', 'provincia', 'escualas_centros_cercanos', 'tipodepropiedad', 
                 'ciudad', 'idzona', 'promedio_precio_hbg_tipo_propiedad_provincia_gen', 'zona', 'idzona']

features = [f for f in features if f not in features_drop]

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features)

In [7]:
x_train.loc[:, x_train.isnull().any()]

""
36242
203272
201436
114481
65169
...
73349
109259
50057
5192


In [14]:
df_encoding = x_train

df_encoding_test = x_test

In [15]:
clf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0, n_jobs=4, verbose=1)
clf.fit(x_train,y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  2.6min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
                      oob_score=False, random_state=0, verbose=1,
                      warm_start=False)

In [1]:
print(categoricos.shape)
print(categoricos_test.shape)
print(df_encoding.shape)
print(df_encoding_test.shape)

set(df_encoding_test.columns)  - set(df_encoding.columns)

NameError: name 'categoricos' is not defined

In [16]:
clf.predict(df_encoding_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished


array([ 886854.02298059, 4913023.22610864, 4868945.26578369, ...,
       3512884.04551905, 3842631.08810592, 6164885.01503134])

In [17]:
clf.feature_importances_

array([1.35355497e-03, 1.66433391e-04, 2.04962547e-04, 1.20557925e-04,
       1.55077113e-02, 1.20791554e-02, 6.02361667e-04, 5.16925337e-04,
       8.14947264e-03, 1.43437910e-05, 1.50070615e-05, 4.12453191e-05,
       3.40209890e-05, 4.26374149e-05, 1.28867078e-02, 1.48003680e-02,
       9.56249352e-04, 8.36903149e-04, 1.57789919e-04, 5.69750083e-04,
       4.09960557e-05, 1.90791516e-03, 1.67078034e-03, 8.89695716e-06,
       3.75424728e-05, 1.21448618e-04, 6.91825046e-05, 2.22774651e-04,
       2.93308432e-04, 5.74782653e-05, 2.87461827e-04, 5.62034255e-04,
       9.51191360e-05, 8.41761811e-03, 6.99529019e-04, 2.77844690e-05,
       3.20948158e-05, 8.03552513e-06, 1.27036298e-04, 4.59406831e-04,
       2.32734437e-04, 1.83687354e-02, 1.32065302e-02, 1.80194759e-02,
       1.37561763e-04, 1.11757225e-04, 5.30560678e-04, 4.65393006e-04,
       4.56909673e-04, 3.06349492e-03, 1.27973432e-03, 1.30742770e-01,
       1.08328647e-01, 2.19119310e-03, 2.84135247e-02, 2.91071199e-03,
      

In [18]:
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df)
        display(df.shape)
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(x_train.columns, clf.feature_importances_):
    feats[feature] = importance #add the name/value pair 

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
full_display(importances.sort_values('Gini-importance', ascending=False))


,Gini-importance
promedio_precio_hbg_tipo_propiedad_provincia,5.455152e-01
promedio_id_zona,1.307428e-01
promedio_id_zona_log,1.083286e-01
varianza_id_zona,2.841352e-02
tam_ambientes,1.836874e-02
metros_cubiertos_normalizados,1.801948e-02
metroscubiertos,1.550771e-02
metroscubiertos_log,1.480037e-02
puntaje,1.420667e-02
metros_totales_normalizados,1.320653e-02


(131, 1)